In [1]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from models import Models
from morph import Morph
from sklearn.preprocessing import LabelEncoder

In [3]:
ECG_datasets = ['AtrialFibrillation', 'CardiacArrhythmia', 'CinCECGTorso',
                'ECG200', 'ECG5000', 'ECGFiveDays', 'NonInvasiveFetalECGThorax1',
                'NonInvasiveFetalECGThorax2', 'StandWalkJump', 'TwoLeadECG','NerveDamage', 'MedicalImages']


df_name = 'CinCECGTorso'
X, y = load_classification(df_name)
print("Shape of X = ", X.shape)

le = LabelEncoder()
y = le.fit_transform(y)
print(f'Classes: {np.unique(y)}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Shape of X =  (1420, 1, 1639)
Classes: [0 1 2 3]


In [ ]:
# Train Model
lstm = Models('lstm', X_train, y_train)
lstm.train_lstm()
# catch = Models('catch22', X_train, y_train)
# catch.train_catch22()
# rocket = Models('rocket', X_train, y_train)
# rocket.train_rocket()

c:\Users\anton\anaconda3\envs\morph\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


In [ ]:
results = {}

for c in np.unique(y):
    res = {}

    mor = Morph(X_test, y_test, c)
    mor.get_DTWGlobalBorderline(perc_samples=0.1)

    res['lstm'] = mor.Binay_MorphingCalculater(lstm)
    res['catch'] = mor.Binay_MorphingCalculater(catch)
    res['rocket'] = mor.Binay_MorphingCalculater(rocket)

    print(c)
    results[c] = res
    columns = ['dataset', 'class', 'model', 'mean', 'std']
    
with open('results/MedicalImages.pkl', 'wb') as f:
    pickle.dump(results, f)

0
1
2
3
4
5
6
7
8
9


In [47]:
with open('results/MedicalImages.pkl', 'rb') as f:
    results = pickle.load(f)

In [48]:
columns = ['dataset', 'class', 'model', 'mean', 'std']

df = []
for c in results.keys():
    for model in results[c].keys():
        data = results[c][model][3]
        line = [df_name, c, model, data['mean'], data['std']]
        df.append(line)

# save to csv
df = pd.DataFrame(df, columns=columns)
df.to_csv('results/medicalimages.csv', index=False)

In [49]:
df.head()

,dataset,class,model,mean,std
0,MedicalImages,0,lstm,0.000000,0.000000
1,MedicalImages,0,catch,0.440040,0.257144
2,MedicalImages,0,rocket,0.451297,0.243854
3,MedicalImages,1,lstm,0.000000,0.000000
4,MedicalImages,1,catch,0.367994,0.217564


In [1]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from models import Models
from morph import Morph
from sklearn.preprocessing import LabelEncoder

results = {}
ECG_datasets = ['AtrialFibrillation', 'CardiacArrhythmia', 'CinCECGTorso',
                'ECG200', 'ECG5000', 'ECGFiveDays', 'NonInvasiveFetalECGThorax1',
                'NonInvasiveFetalECGThorax2', 'StandWalkJump', 'TwoLeadECG','NerveDamage', 'MedicalImages']


for df_name in ECG_datasets:
    try:
        X, y = load_classification(df_name)
        le = LabelEncoder()
        y = le.fit_transform(y)
    except:
        print(df_name, 'Dataset Not Available')
        continue


    if X.shape[1] != 1:
        print(df_name, 'Multivariate Time Series')
        continue

    print('-'*60)
    print(df_name)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Train Models
    lstm = Models('lstm', X_train, y_train)
    lstm.train_lstm()
    catch = Models('catch22', X_train, y_train)
    catch.train_catch22()
    rocket = Models('rocket', X_train, y_train)
    rocket.train_rocket()

    # Loop through class
    for c in np.unique(y):
        res = {}

        mor = Morph(X_test, y_test, c)
        mor.get_DTWGlobalBorderline(perc_samples=0.1)

        res['lstm'] = mor.Binay_MorphingCalculater(lstm)
        res['catch'] = mor.Binay_MorphingCalculater(catch)
        res['rocket'] = mor.Binay_MorphingCalculater(rocket)

        print(c)
        results[c] = res
        
        with open('results/experiments.pkl', 'wb') as f:
            pickle.dump(results, f)


    with open('results/experiments.pkl', 'rb') as f:
        data = pickle.load(f)
    
    df = []
    for c in results.keys():
        for model in results[c].keys():
            data = results[c][model][3]
            line = [df_name, c, model, data['mean'], data['std']]
            df.append(line)

    # save to csv
    columns = ['dataset', 'class', 'model', 'mean', 'std']
    df = pd.DataFrame(df, columns=columns)
    df.to_csv('results/medicalimages.csv', index=False)


AtrialFibrillation Multivariate Time Series
CardiacArrhythmia Dataset Not Available
------------------------------------------------------------
CinCECGTorso


c:\Users\anton\anaconda3\envs\morph\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


KeyboardInterrupt: 